# Opus Method 2

<img src="images/opus.png" alt="drawing" width="200"/>

This method of Opus verifying information about an indvidual relies on passing the user a unique key and that user posting that key on their social media profile. Opus can then parse the account page containing the key and any subsequent data on that page. Opus knows this page belongs to the user because only the user can post to that page.

In order to issue a credential, first we need to define it on the ledger:
<img src="images/opus_1A.png" alt="drawing" width="500"/>

## 0. Define Scheme on Ledger


### 0.1 Connect to Opus Agent with Controller as Opus

This tutorial assumes a PeerDiDComm connection has been established between the Opus Agent and the User Agent

TODO: Update docker and manage files to make this the Opus Agent in name also


In [1]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

### 0.2 Write Scheme to Ledger

In [2]:
# Define you schema name - must be unique on the ledger
schema_name = "openmined_member"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["Username", "OpenMined Member"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

gZj39Y8JGQ5GVQp2y6zFH:2:openmined_member:0.0.1


In [3]:
schema = await agent_controller.schema.get_by_id(schema_id)
print(schema)

{'schema': {'ver': '1.0', 'id': 'gZj39Y8JGQ5GVQp2y6zFH:2:openmined_member:0.0.1', 'name': 'openmined_member', 'version': '0.0.1', 'attrNames': ['Username', 'OpenMined Member'], 'seqNo': 10}}


### 0.3 Write Credential to Ledger

More details in the [definitions notebook](http://localhost:8888/notebooks/definitions_api.ipynb)

**Note: Again this can only be done once per issuer, per schema_id.**

In [4]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

gZj39Y8JGQ5GVQp2y6zFH:3:CL:10:default


### 0.4 Get Schema on Ledger by ID

In [5]:
schema = await agent_controller.schema.get_by_id(schema_id)
print(schema)

{'schema': {'ver': '1.0', 'id': 'gZj39Y8JGQ5GVQp2y6zFH:2:openmined_member:0.0.1', 'name': 'openmined_member', 'version': '0.0.1', 'attrNames': ['Username', 'OpenMined Member'], 'seqNo': 10}}


Next we need to implement a method for moving <b>trusted</b> data from third parties online into our private wallets. Method 2 can be seen below. This good trust guarentees as Opus is able to verify that the html page presented by the user has the authority to post on the account page of the third party service. However, this limits credentialing to only public information. The user doesn't need to trust Opus but Opus is limited to publicly available infromation when it comes to what it can sign off.

<img src="images/opus_2B.png" alt="drawing" width="800"/>

## 1. User Retrieves Ownership Token

Handle the message, prepare a response with a unique string for account verification. User posts it to their profile.

### 1.0 Check for Active connection with User Agent

In [6]:
response = await agent_controller.connections.get_connections()
results = response['results']
# print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
#     print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")

Active Connection ID :  bf101bf7-6c10-4499-8bc3-1c65082169d0


### 1.1 Setup a listener for the basicmessages topic

This is emitted using PyPubSub when the controller receives a basicmessages webhook from the agent. This happens everytime the agent receives a basicmessage.

In [7]:
%autoawait
import time
import asyncio

def messages_handler(payload):
    connection_id = payload["connection_id"]
#     USER = payload["content"]
    print("USER RESPONSE:", payload["content"], "\nCONNECTION_ID: "+connection_id)

message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([cred_listener, message_listener], defaults=True)

IPython autoawait is `on`, and set to use `asyncio`


## --- Run through steps 1-3 on User side ---

### 1.2 Generate Verification Code


In [9]:
import secrets

# ownership_proof = secrets.token_hex(16)
ownership_proof = "53a4198707658f2b0402af57441aa380"
ownership_statement = "#OPUS "+ownership_proof+"=="
print(ownership_statement)

#OPUS 53a4198707658f2b0402af57441aa380==


### 1.3 Send Verification Code over Secure Channel

In [10]:
basic_message = "Post '"+ownership_statement+"' on your Github profile. When ready, respond with your username."
response = await agent_controller.messaging.send_message(connection_id, basic_message)
print("BASIC MESSAGE - Opus -> Alice")
print(response)

BASIC MESSAGE - Opus -> Alice
{}
USER RESPONSE: H4LL 
CONNECTION_ID: bf101bf7-6c10-4499-8bc3-1c65082169d0


## 2. Sign in to Account and Post Ownership Token

Performed on the user side. User then sends username to Opus. 

In [11]:
#TODO Retrieve Username from Aries message
USER = "H4LL"

## 3. Parse Account Page URL, Retrieve Ownership Token

Crawling is not yet robust but should complete given the user has posted to their profile. 

In [12]:
import requests
import re
from bs4 import BeautifulSoup 

#Retrieve Page
http_response = requests.get("https://github.com/"+USER)

#Parse Account info for ownership token
soup = BeautifulSoup(http_response.text)
ownership_token = soup.findAll("div", {"class": "user-profile-bio"})[0].findAll("div")[0].text.strip()
ownership_token = ownership_token.split("#OPUS ",1)[1].split("==",1)[0] 

if ownership_token == ownership_proof:
    print("Account Ownership Verified")
else:
    print("Unable to Link")

Account Ownership Verified


## 4. Parse Authenticated Page for Organisation Membership

Here we parse the recieved organisation page (as Opus) and recieve the membership affiliations.

In [13]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(http_response.text)

orgsSection = soup.findAll("div", {"class": "border-top pt-3 mt-3 clearfix hide-sm hide-md"})[0].findAll('img')
myOrgs = set(tag['alt'] for tag in orgsSection)

if '@OpenMined' in myOrgs:
    print("Account Linked to OpenMined")
else:
    print("Unable to Find OpenMined org")

Account Linked to OpenMined


## 5. Issue Parsed Data as OpenMined Credential

### 5.3 Populate the Attribues to Issue to Alice

In [14]:
credential_attributes = [
    {"name": "Username", "value": "H4LL"},
    {"name": "OpenMined Member", "value": "1"}
]
print(credential_attributes)

[{'name': 'Username', 'value': 'H4LL'}, {'name': 'OpenMined Member', 'value': '1'}]


### 5.5 Send credential

In [15]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

{'issuer_did': 'gZj39Y8JGQ5GVQp2y6zFH', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'Username', 'value': 'H4LL'}, {'name': 'OpenMined Member', 'value': '1'}]}, 'connection_id': 'bf101bf7-6c10-4499-8bc3-1c65082169d0', 'trace': False, 'comment': '', 'cred_def_id': 'gZj39Y8JGQ5GVQp2y6zFH:3:CL:10:default', 'schema_id': 'gZj39Y8JGQ5GVQp2y6zFH:2:openmined_member:0.0.1', 'schema_name': 'openmined_member', 'schema_version': '0.0.1', 'schema_issuer_did': 'gZj39Y8JGQ5GVQp2y6zFH'}
Credential exchange 2ccf104f-e0ac-45ef-81e3-7a81deab9284, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 2ccf104f-e0ac-45ef-81e3-7a81deab9284, role: issuer, state: offer_sent
Offering: [{'name': 'Username', 'value': 'H4LL'}, {'name': 'OpenMined Member', 'value': '1'}]
Handle Credentials
Credential exchange 2ccf104f-e0ac-45ef-81e3-7a81deab9284, role: issuer, state: request_received
Offe

### 5.6 Finish the tutorial from the Users notebook